In [ ]:
'''
Detecting Data Fraud Case Study
'''
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_json('/home/david/Documents/temp/data.json')
df.head()

In [ ]:
df.acct_type.value_counts()

In [ ]:
#df.info()

In [ ]:
df['fraud'] = np.where((df.acct_type == 'fraudster_event')
                       | (df.acct_type == 'fraudster')
                       | (df.acct_type == 'fraudster_att')
                       , 1
                       , 0
                      )
df.head()

In [ ]:
df.fraud.value_counts()

In [ ]:
# downsample
df = df.groupby('fraud').apply(lambda x: x.sample(n=1293)).reset_index(drop=True)
df.head()

In [ ]:
df.shape

In [ ]:
df.fraud.value_counts()

In [ ]:
df_fraud = df[df.fraud == 1]
df_fraud.shape

In [ ]:
df_true = df[df.fraud != 1]
df_true.shape

In [ ]:
df.columns

In [ ]:
df.iloc[3]


In [ ]:
df.description.iloc[0] # use beautiful soup

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
soup = BeautifulSoup(df.description.iloc[0])

In [ ]:
soup.find_all('p')[0].get_text()

In [ ]:
texts = []
for text in soup.find_all('p'):
    texts.append(text.get_text())

In [ ]:
texts

In [ ]:
text2_str = ' '.join(texts)
text2_str

### Make a function to convert description into string

In [ ]:
def to_string(des):
    
    from bs4 import BeautifulSoup
    import string, re
    from num2words import num2words as n2w # may have to download this, terminal: pip3 install num2words
    
    soup = BeautifulSoup(des)
    texts = []
    for text in soup.find_all('p'): # gets all the words in paragraphs 
        texts.append(text.get_text())
    texts = ' '.join(texts).lower() # joins sentences together into one long paragraphs
    texts = texts.replace('$', 'dollar ') # change $ symbole to 'dollar'
    texts = re.split(r'\W+', texts) # keeps all the words and numbers only, drops all symbol characters 
    
    new_words = []
    for word in texts: 
        if word[-2:] == 'am' or word[-2:] == 'pm': # split am and pm from numbers
            new_words.append(word[:-2]) 
            new_words.append(word[-2:])
        else:
            new_words.append(word)
    
    texts = new_words
    
    new_words = []
    for word in texts:
        if word.isdigit(): # changes numbers into words 200 becomes two hundred 
            new_words.append(n2w(int(word)))
        else:
            new_words.append(word)
            
    texts = new_words
    
    new_words = []
    for word in texts:
        if ' ' in word:
            new_words.extend(word.split(' '))
        else:
            new_words.append(word)
    
    return ' '.join(new_words)

In [ ]:
to_string(df.description.iloc[2])

In [ ]:
# text2 = to_string(df.description.iloc[2]).lower()
# text2

In [ ]:
# text2 = text2.replace('$', 'dollar ')
# text2

In [ ]:
# import re
# words = re.split(r'\W+', text2)
# words

In [ ]:
# new_words = []
# for word in words:
#     if word[-2:] == 'am' or word[-2:] == 'pm':
#         new_words.append(word[:-2])
#         new_words.append(word[-2:])
#     else:
#         new_words.append(word)
# new_words

In [ ]:
# from num2words import num2words as n2w

# new_words_text_only = []
# for word in new_words:
#     if word.isdigit():
#         new_words_text_only.append(n2w(int(word)))
#     else:
#         new_words_text_only.append(word)
# new_words_text_only

In [ ]:
# final_word_lst = []
# for word in new_words_text_only:
#     if ' ' in word:
#         final_word_lst.extend(word.split(' '))
#     else:
#         final_word_lst.append(word)
# final_word_lst

In [ ]:
string_only = ''.join(to_string(df.description.iloc[2]))
string_only

In [ ]:
letter_dict = {}
for letter in string_only:
    if letter in letter_dict:
        letter_dict[letter] += 1
    else:
        letter_dict[letter] = 1


vals = []
for x in letter_dict.values():
    vals.append(x)


first_vals_dict = {}
for x in vals:
    if int(str(x)[0]) in first_vals_dict:
        first_vals_dict[int(str(x)[0])] += 1
    else:
        first_vals_dict[int(str(x)[0])] = 1

sorted_firsts = sorted(first_vals_dict.items())
sorted_firsts

In [ ]:
x = [x[0] for x in sorted_firsts]
y = [x[1] for x in sorted_firsts]
print(x)
print(y) # use this as a table to model 

In [ ]:
# look up different ways to normalize: min, max
# look up: log trap. area under the curve rules 

In [ ]:
plt.bar(x,y);

In [ ]:
lst_firsts = []
for i in range(len(x)):
    for itr in range(y[i]):
        lst_firsts.append(x[i])
lst_firsts

In [ ]:
def median(lst):
    n = len(lst)
    if n%2 == 0:
        return round((lst[n//2 -1] + lst[n//2])/2, 1)
    else: 
        return lst[n//2]

In [ ]:
median(lst_firsts)

In [ ]:
to_string(df.description.iloc[2])

### Add 'desc_string' column of texts from description column

In [ ]:
df['desc_string'] = df.description.apply(to_string)

### Identify non-english entries, pip install langdetect

In [ ]:
#from langdetect import detect

In [ ]:
# def is_en(txt):
#     try:
#         return ld.detect(txt)!='en'
#     except:
#         return False

# df_foreign = df[df['desc_string'].apply(is_en)]
# df_foreign

### Translate to English using googletrans library: pip install googletrans: https://py-googletrans.readthedocs.io/en/latest/

In [ ]:
from googletrans import Translator

In [ ]:
translations = Translator()

In [ ]:
type(df.desc_string.iloc[0])

In [ ]:
translations.detect(df.desc_string.iloc[0]).lang

In [ ]:
translations.detect('i am typing in english').lang

In [ ]:
def lang(txt):
    return translations.detect(txt).lang

In [ ]:
def to_eng(txt):
    if translations.detect(txt).lang != 'en':
        return translations.translate(txt, dest = 'en').text
    else:
        return txt

In [ ]:
lang('yo quero mucho')

In [ ]:
translations.translate('yo quero mucho', dest = 'en').text

In [ ]:
to_eng('yo quero mucho')

### Create language column that specify the language source

In [ ]:
df.shape

In [ ]:
df['desc_lang'] = df.desc_string.apply(lang)

In [ ]:
df.desc_lang.value_counts()

In [ ]:
df['desc_eng'] = df.desc_string.apply(to_eng)

In [ ]:
df[df.desc_lang != 'en']['desc_eng']

In [ ]:
to_eng(df.desc_eng.iloc[2521])

In [ ]:
# df.desc_string.iloc[280]

### TF-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
words1 = to_string(df.description.iloc[2])
words2 = to_string(df.description.iloc[3])
words3 = to_string(df.description.iloc[4])

In [ ]:
word_lst = [words1
            , words2
            , words3
           ]
tf_model = TfidfVectorizer()
vectors = tf_model.fit_transform(word_lst)
vectors[0].T.todense()

In [ ]:
df.head()

In [ ]:
df.desc_string.iloc[0]

In [ ]:
# the following needs up to 55.5 gbs of memory!
# this is the reson for downsampling 

In [ ]:
tf_model = TfidfVectorizer()
row_vectors = tf_model.fit_transform(df.desc_string)

word_columns = tf_model.get_feature_names()

dense = row_vectors.todense()

dense_list = dense.tolist()


In [ ]:
df_tfidf = pd.DataFrame(dense_list, columns = word_columns)

In [ ]:
df_tfidf.shape

In [ ]:
df

In [ ]:
df_tfidf.shape

In [ ]:
df.shape

In [ ]:
df_tfidf = df_tfidf.assign(f = df.fraud)

In [ ]:
df_tfidf.shape

In [ ]:
df_tfidf

In [ ]:
# resample to scramble 
df_tfidf = df_tfidf.sample(frac=1) 

In [ ]:
df_tfidf

### Random Forest Classifier

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X = df_tfidf.drop('f', axis=1)
y = df_tfidf.f
X_train, X_test, y_train, y_test = train_test_split(X
                                                    , y
                                                    , test_size = 0.25
                                                    , random_state = 42
                                                   )

In [ ]:
X_train.shape

In [ ]:
X_test.shape

In [ ]:
y_train.shape

In [ ]:
y_test.shape

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rf_model = RandomForestClassifier(n_estimators = 199
                                  , bootstrap = True
                                  , max_features = 'sqrt'
                                 )

rf_model.fit(X_train, y_train)

In [ ]:
yhat = rf_model.predict(X_test)
rf_probs = rf_model.predict_proba(X_test)[:, 1]

In [ ]:
# ROC AUC Score

from sklearn.metrics import roc_auc_score

roc_value = roc_auc_score(y_test, rf_probs)
roc_value

In [ ]:
# feature importances

fi = pd.DataFrame({'word': list(X_train.columns)
                   , 'importance': rf_model.feature_importances_
                  }
                 ).sort_values('importance', ascending = False)
fi.head()

In [ ]:
# To Do: Need to Filter out Stop words, upsample, use cosine similarity (make a new column)
# Word to vec, google algorithm 
# Any new descriptions will need to be vectorized, must be in the test set 
# class weight = balanced <-- use as hyperparameter 
# SMOTE method for upsampling 
# minimize false negatives (predicted non-frauds but are actually fraud) 
# check precision recall score, recall preferred (proportions of actual positives predicted)
# F1 score (to maximize precision and recall scores)

### Check for Benford's Distribution

In [ ]:

body_lengths_true = df_true.body_length.values
int(str(body_lengths_true[1])[0])

In [ ]:
body_lengths_true_first = [int(str(x)[0]) for x in body_lengths_true]

In [ ]:
plt.hist(body_lengths_true_first)
plt.ylabel('Frequency')
plt.xlabel('First Number')
plt.title('Non-Fraud');

In [ ]:
body_lengths_fraud = df_fraud.body_length.values
body_lengths_fraud_first = [int(str(x)[0]) for x in body_lengths_fraud]

In [ ]:
plt.hist(body_lengths_fraud_first)
plt.ylabel('Frequency')
plt.xlabel('First Number')
plt.title('Fraud');

In [ ]:
# class imbalance, do over or under sampling. 
# vectorize the words and then add body length to train model 
# use tf-idf, count vectorizer to train the model 